In [1]:
import os
import sys

import pandas as pd

FIGURE_DIR = os.getcwd()
MIL_DIR = os.path.dirname(FIGURE_DIR)
NB_DIR = os.path.dirname(MIL_DIR)
ROOT_DIR = os.path.dirname(NB_DIR)
sys.path.append(ROOT_DIR)

### build dataframe from dataset

In [2]:
from core.data_model import *
from utils.serialization_ops import load_pickle

patient_dataset: PatientDataSet = load_pickle(os.path.join(ROOT_DIR, "dataset_positive_negative.pickle"))

In [3]:
# Annotation gender
import tqdm
from core.dynamodb_ops import DynamoDBClient
dynamodb_client = DynamoDBClient(os.path.join(ROOT_DIR, "data", "keyfile.yaml"))

patient_container = list()
for patient_data in patient_dataset:
    try:
        demographics = dynamodb_client.get_clinical_info(patient_data.sample_id)
        patient_data.gender = demographics["gender"]
        patient_data.symptoms = [Symptom(*symptom) for symptom in demographics["symptoms"]]
        patient_container.append(patient_data)
    except:
        continue

patient_dataset = PatientDataSet(patient_container)

hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo ids for patient symptoms are missing. (here's queried symptoms [])
hpo id

In [4]:
from collections import defaultdict 

patient_info = defaultdict(list)

onset_order = {
    'Antenatal':0 , 
    'Neonatal': 1, 
    'Infancy': 2,
    'Childhood': 3,
    "Adolescent": 4,
    'Adult': 5, 
    'Elderly': 6, 
    'Unknown': 7
}
idx_to_sym = { x: y for y, x in onset_order.items()}

# get patient's clinical info from dataset object
all_pheno_count = defaultdict(int)
for p_data in patient_dataset:

    patient_info["is_reported"].append(p_data.bag_label)
    patient_info["id"].append(p_data.sample_id)
    patient_info["gender"].append(p_data.gender)

    ealiest_onset_idx = min([onset_order.get(x.onsetAge, 7) for x in p_data.symptoms])
    patient_info["ealiest_onset"].append(idx_to_sym[ealiest_onset_idx])
    patient_info["n_pheno"].append(len(p_data.symptoms))
    
    for sym in  p_data.symptoms:
        all_pheno_count[sym.hpo] += 1

    patient_info["snv_confirmed"].append(True if p_data.snv_data.y.sum() >=1 else False)
    patient_info["cnv_confirmed"].append(True if p_data.cnv_data.causal_variant else False)
        
# build dataframe with gathered data
patient_info_df = pd.DataFrame(patient_info)
patient_info_df.loc[patient_info_df["id"] == "EPN22-EBAB", "gender"] = "female"
patient_info_df = patient_info_df.loc[
    ~((patient_info_df["is_reported"] == False) & \
      (patient_info_df["cnv_confirmed"] | patient_info_df["snv_confirmed"]))
]
patient_info_df["both_confirmed"] = (patient_info_df["cnv_confirmed"] & patient_info_df["snv_confirmed"])

# dtype
patient_info_df["n_pheno"] = patient_info_df["n_pheno"].astype("int16")


In [5]:
import math
from scipy import stats
from scipy.stats import chi2_contingency

# define Cohen'd and Cramers'V
def cohen_d(mean1, mean2, sd1, sd2, n1, n2):
    pooled_sd = math.sqrt(((sd1 ** 2) * (n1 - 1) + (sd2 ** 2) * (n2 - 1)) / (n1 + n2 - 2))
    d = (mean1 - mean2) / pooled_sd
    return d

def cramers_v(confusion_matrix):
    confusion_matrix = np.array(confusion_matrix)
    chi2, _, _, _ = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum()
    min_dim = min(confusion_matrix.shape) - 1  # Minimum of rows/columns minus 1
    v = np.sqrt(chi2 / (n * min_dim))
    return v

### Test reported goup versus Non reportable group
- t-test and Cohen's d for comparing mean
- chi-square test and cramer'V for assessing association b.t. categorival variables. 

In [6]:
def test_for_cate_var(table):
    # apply Yates' correction if dof == 1
    correction =  max(table.shape) == 2
    chi_ct = chi2_contingency(
        table, correction=correction
    )
    print("chi2-statistic:", chi_ct.statistic)
    print("P-value:", chi_ct.pvalue)
    print("Cramer'V:", cramers_v(table))

def make_pretty_table(table):
    table.columns = ["non-reportable", "reported"]
    table_pretty = table.copy(deep=True)

    add_commas = lambda x: '{:,}'.format(x)
    for col, denominator in [("non-reportable", 13903), ("reported", 10536)]:
        percent = table_pretty[col] * 100 / denominator
        attach_percent = percent.apply(lambda x: f"({x:.3f})")
        table_pretty[col] = table_pretty[col].apply(add_commas) + attach_percent

    return table_pretty

In [19]:
from tableone import TableOne
table1 = TableOne(
    patient_info_df, 
    columns=["gender", "ealiest_onset", "n_pheno", "snv_confirmed", "cnv_confirmed", "both_confirmed"], 
    categorical=["gender", "ealiest_onset", "snv_confirmed", "cnv_confirmed", "both_confirmed"],
    groupby="is_reported", 
    decimals=3,
    missing=False,
    overall=False, 
    pval=True
)
table1.to_csv("table1.csv")
table1

Grouped by is_reported                       
                                                  False           True P-Value
n                                                 14998           8157        
gender, n (%)         female              6484 (43.232)  3674 (45.041)   0.008
                      male                8514 (56.768)  4483 (54.959)        
ealiest_onset, n (%)  Adolescent            877 (5.847)    328 (4.021)  <0.001
                      Adult               5050 (33.671)  1374 (16.844)        
                      Antenatal             673 (4.487)    406 (4.977)        
                      Childhood           2528 (16.856)  1591 (19.505)        
                      Elderly               133 (0.887)     14 (0.172)        
                      Infancy             2535 (16.902)  1959 (24.016)        
                      Neonatal            3027 (20.183)  2282 (27.976)        
                      Unknown               175 (1.167)    203 (2.489)        
n_pheno, mean (SD)                        3.442 (3.481)  4.438 (4.347)  <0.001
snv_confirmed, n (%)  False             14998 (100.000)    326 (3.997)  <0.001
                      True                               7831 (96.003)        
cnv_confirmed, n (%)  False             14998 (100.000)  7765 (95.194)  <0.001
                      True                                 392 (4.806)        
both_confirmed, n (%) False             14998 (100.000)  8091 (99.191)  <0.001
                      True                                  66 (0.809)

In [20]:
contingency_table = pd.crosstab(patient_info_df['is_reported'], patient_info_df['gender'])

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Calculate Cramer's V
n = contingency_table.sum().sum()
phi2 = chi2 / n
cramersv = (phi2 / min(contingency_table.shape[0]-1, contingency_table.shape[1]-1))**0.5

# Print the results
print("Chi-squared test statistic:", chi2)
print("P-value:", p)
print("Cramer's V:", cramersv)

Chi-squared test statistic: 6.945505612408337
P-value: 0.008403008296897241
Cramer's V: 0.017319268320795497


In [21]:
reported_group  = patient_info_df[patient_info_df["is_reported"]]
non_reported_group  = patient_info_df[~patient_info_df["is_reported"]]

n_pheno_reported = reported_group["n_pheno"].values
n_pheno_non_reported = non_reported_group["n_pheno"].values

cohend = cohen_d(n_pheno_reported.mean(), n_pheno_non_reported.mean(), 
        n_pheno_reported.std(), n_pheno_non_reported.std(), 
        len(n_pheno_reported), len(n_pheno_non_reported)) 

print("Test result for n_pheno")
print("Cohen's d:", cohend)

Test result for n_pheno
Cohen's d: 0.2616710105807334


In [22]:
# for analysis phenotypes with HPO 

## load HPO ontology
from pronto import Ontology
import warnings
import pronto
warnings.filterwarnings("ignore", category=pronto.warnings.ProntoWarning)

onto = Ontology("https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2023-10-09/hp.owl")

/tmp/ipykernel_212498/1147481039.py:9: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  onto = Ontology("https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2023-10-09/hp.owl")


In [23]:
# child of HP:0000118(Phenotypic abnormality) with dist 1
phenotype_groups = list(onto["HP:0000118"].subclasses(distance=1, with_self=False).to_set().ids)

# count frequency of each pheno group in dataset
phenotype_groups_count = defaultdict(int)
for pheno_id, counts in all_pheno_count.items():
    for group_id in phenotype_groups:
        super_class_set = onto[pheno_id].superclasses().to_set().ids
        if group_id in super_class_set:
            phenotype_groups_count[group_id] += counts

In [24]:
print("Frequency for each group in whole dataset", "\n")
phenotype_groups_sorted = dict(sorted(phenotype_groups_count.items(), key=lambda x: x[1], reverse=True))
for key, val in phenotype_groups_sorted.items():
    print(f"{onto[key].name}: {val:,}")

Frequency for each group in whole dataset 

Abnormality of the nervous system: 24,947
Abnormality of the musculoskeletal system: 14,011
Abnormality of head or neck: 12,074
Abnormality of the cardiovascular system: 8,530
Abnormality of the eye: 7,217
Growth abnormality: 4,575
Abnormality of the integument: 4,570
Abnormality of limbs: 4,547
Abnormality of metabolism/homeostasis: 4,476
Abnormality of the genitourinary system: 4,272
Abnormality of the ear: 3,136
Abnormality of the digestive system: 2,994
Abnormality of the immune system: 2,916
Abnormality of blood and blood-forming tissues: 2,312
Abnormality of the endocrine system: 2,077
Abnormality of the respiratory system: 1,383
Constitutional symptom: 951
Neoplasm: 833
Abnormality of prenatal development or birth: 563
Abnormal cellular phenotype: 289
Abnormality of the breast: 212
Abnormality of the voice: 92
Abnormality of the thoracic cavity: 5


In [25]:
# the 5 most prevalence groups
prevalence_groups = ["HP:0000707", "HP:0033127", "HP:0000152", "HP:0001626", "HP:0000478"]

# categorize each patient's symptoms
symptoms_categorized = []
for p_data in patient_dataset:

    # initialize row
    row = {
        term: False
        for term in prevalence_groups
    }
    row["etc"] = False
    row["patient_id"] = p_data.sample_id  
    row["is_reported"] = p_data.bag_label

    for hpo_id in p_data.all_hpos:
        super_class_set=  onto[hpo_id].superclasses(with_self=True).to_set().ids
        phenotype_category = super_class_set.intersection(prevalence_groups)
        if len(phenotype_category) != 0:
            term_id = list(phenotype_category)[0]
            row[term_id] = True 
        else:
            row["etc"] = True 
    symptoms_categorized.append(row)

symptoms_categorized = pd.DataFrame(symptoms_categorized)

In [26]:
# test for symptom count
contingency_table = symptoms_categorized.groupby("is_reported").agg("sum").T
contingency_table = contingency_table.loc[contingency_table.index.str.startswith("HP:") | contingency_table.index.str.startswith("etc")]
make_pretty_table(contingency_table).to_csv("table1.phenotype.csv")
print(make_pretty_table(contingency_table), "\n")
print("Test result for ealiest_onset\n")
test_for_cate_var(contingency_table.values)

           non-reportable       reported
HP:0000707  5,627(40.473)  3,559(33.779)
HP:0033127  3,163(22.750)  2,423(22.997)
HP:0000152  2,618(18.830)  2,272(21.564)
HP:0001626  4,306(30.972)  1,601(15.196)
HP:0000478  2,253(16.205)  2,270(21.545)
etc         8,047(57.880)  4,607(43.726) 

Test result for ealiest_onset

chi2-statistic: 783.4230097140459
P-value: 4.459952514278462e-167
Cramer'V: 0.13537872536751383
